In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re, json, itertools
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
!pip install openpyxl
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
df = pd.read_excel('posts.xlsx')

In [7]:
df.shape

(39362, 22)

In [8]:
df = df[['Attribute:Body', 'Attribute:Tags']]
df.columns = ['Body', 'Tags']
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df = df.iloc[:10000, :]

In [9]:
df.head(5)

,Body,Tags
0,<p>I often hear about subatomic particles havi...,<quantum-mechanics><particle-physics><angular-...
1,<p>How would you explain string theory to non-...,<string-theory><education>
2,<p>This is a question that has been posted at ...,<particle-physics><group-theory><representatio...
3,<p>What are the main problems that we need to ...,<quantum-mechanics><quantum-interpretations><h...
4,<p>Hamilton's principle states that a dynamic ...,<lagrangian-formalism><variational-principle><...


In [10]:
df.shape

(10000, 2)

In [11]:
#preprocessing the Body
def clean_text(sentence):
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, '', sentence)
    pattern = re.compile(r'[^a-z]+')
    sentence = sentence.lower()
    sentence = pattern.sub(' ', sentence).strip()
    word_list = word_tokenize(sentence)
#     stopwords_list = set(stopwords.words('english'))
#     word_list = [word for word in word_list if word not in stopwords_list]
#     word_list = [word for word in word_list if len(word)>1]
#     ps = PorterStemmer()
#     word_list = [ps.stem(word) for word in word_list]
    sentence = ' '.join(word_list)
    return sentence
df['Body'] = df['Body'].apply(lambda x: clean_text(x))

In [12]:
df.head()

,Body,Tags
0,i often hear about subatomic particles having ...,<quantum-mechanics><particle-physics><angular-...
1,how would you explain string theory to non phy...,<string-theory><education>
2,this is a question that has been posted at man...,<particle-physics><group-theory><representatio...
3,what are the main problems that we need to sol...,<quantum-mechanics><quantum-interpretations><h...
4,hamilton s principle states that a dynamic sys...,<lagrangian-formalism><variational-principle><...


In [15]:
#creating one-hot representation for all the tags
df_tags = pd.DataFrame(np.zeros((df.shape[0], len_tags), dtype=int), columns=tags)
for i in range(len_rows):
    for x in df.Tags[i].split():
        df_tags[x][i] = 1
df = pd.concat([df, df_tags], axis=1)
df.head()

,Body,Tags,1pi-effective-action,absolute-units,absorption,acceleration,accelerator-physics,accretion-disk,acoustics,action,adiabatic,adm-formalism,ads-cft,aerodynamics,aether,affine-lie-algebra,air,aircraft,algebraic-geometry,algebraic-topology,algorithms,amorphous-solids,analyticity,angular-momentum,angular-velocity,antennas,anthropic-principle,anti-de-sitter-spacetime,anticommutator,antimatter,anyons,applied-physics,approximations,arrow-of-time,asteroids,astrometrics,astronomy,astrophotography,astrophysics,asymptotics,...,variational-calculus,variational-principle,vector-fields,vectors,velocity,vibrations,virial-theorem,virtual-particles,viscosity,visible-light,vision,visualization,voltage,volume,vortex,warp-drives,water,wave-particle-duality,wavefunction,wavefunction-collapse,waveguide,wavelength,waves,weak-interaction,weather,weight,white-dwarfs,white-holes,wick-rotation,wick-theorem,wightman-fields,wigner-eckart,wigner-transform,wilson-loop,wimps,work,wormholes,x-ray-crystallography,x-rays,yang-mills
0,i often hear about subatomic particles having ...,quantum-mechanics particle-physics angular-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,how would you explain string theory to non phy...,string-theory education,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,this is a question that has been posted at man...,particle-physics group-theory representatio...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,what are the main problems that we need to sol...,quantum-mechanics quantum-interpretations h...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hamilton s principle states that a dynamic sys...,lagrangian-formalism variational-principle ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#list of all tags
def clean_tag(line):
    line = re.sub('[<>]', ' ', line)
    return line
df.Tags = df.Tags.apply(lambda x: clean_tag(x))
vectorizer = CountVectorizer(tokenizer= lambda text : text.split(" "))
tag_dtm = vectorizer.fit_transform(df["Tags"])
tags = vectorizer.get_feature_names()
tags = [x for x in tags if x]
len_rows = df.shape[0]
len_tags = len(tags)
print('Total no. of unique tags: ',len_tags)
print('dataframe size: ', len_rows)
tags_count = {}
temp_series = df_tags.sum().sort_values(ascending=False)
for i in range(len_tags):
    tags_count[temp_series.index[i]] = temp_series.values[i]

Total no. of unique tags:  723
dataframe size:  10000


In [17]:
# !wget 'https://nlp.stanford.edu/data/glove.6B.zip'
# !unzip 'glove.6B.zip'

In [4]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [18]:
tags[:10]

['1pi-effective-action',
 'absolute-units',
 'absorption',
 'acceleration',
 'accelerator-physics',
 'accretion-disk',
 'acoustics',
 'action',
 'adiabatic',
 'adm-formalism']

In [19]:
#getting tag embeddings
tag_embeddings = []
count = 0
for tag in tags:
    sum = np.zeros(100)
    try:
        sub_tags = tag.split('-')
        for x in sub_tags:
            try:
                sum += embeddings_index[x]
            except:
                continue
        tag_embeddings.append(sum/len(sub_tags))
    except:
        count+=1
        print(tag)
tag_embeddings = np.array(tag_embeddings)

In [20]:
df.head()

,Body,Tags,1pi-effective-action,absolute-units,absorption,acceleration,accelerator-physics,accretion-disk,acoustics,action,adiabatic,adm-formalism,ads-cft,aerodynamics,aether,affine-lie-algebra,air,aircraft,algebraic-geometry,algebraic-topology,algorithms,amorphous-solids,analyticity,angular-momentum,angular-velocity,antennas,anthropic-principle,anti-de-sitter-spacetime,anticommutator,antimatter,anyons,applied-physics,approximations,arrow-of-time,asteroids,astrometrics,astronomy,astrophotography,astrophysics,asymptotics,...,variational-calculus,variational-principle,vector-fields,vectors,velocity,vibrations,virial-theorem,virtual-particles,viscosity,visible-light,vision,visualization,voltage,volume,vortex,warp-drives,water,wave-particle-duality,wavefunction,wavefunction-collapse,waveguide,wavelength,waves,weak-interaction,weather,weight,white-dwarfs,white-holes,wick-rotation,wick-theorem,wightman-fields,wigner-eckart,wigner-transform,wilson-loop,wimps,work,wormholes,x-ray-crystallography,x-rays,yang-mills
0,i often hear about subatomic particles having ...,quantum-mechanics particle-physics angular-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,how would you explain string theory to non phy...,string-theory education,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,this is a question that has been posted at man...,particle-physics group-theory representatio...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,what are the main problems that we need to sol...,quantum-mechanics quantum-interpretations h...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hamilton s principle states that a dynamic sys...,lagrangian-formalism variational-principle ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
#importing libraries
import tensorflow as tf
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.activations import sigmoid
from keras.layers import Input, Dense, Flatten, Dropout, Embedding, GlobalMaxPooling1D, dot, Reshape, Layer
from tensorflow.keras.backend import variable, dot as k_dot, sigmoid, relu
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import Variable
# from spektral.layers import GCNConv

from sklearn.model_selection import train_test_split

### Embedding of questions

In [22]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def max_length(lines):
    return max([len(s.split()) for s in lines])

def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [23]:
X = df['Body']
y = df.drop(['Body', 'Tags'], axis=1).values

In [24]:
tokenizer = create_tokenizer(X)
length = max_length(X)
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
X = encode_text(tokenizer, X, length)
print(X.shape)

Max document length: 3325
Vocabulary size: 24280
(10000, 3325)


In [25]:
def get_emb_matrix(embeddings_index, vocab_size, emb_size = 100):
    """ Creates embedding matrix from word vectors"""
    vocab_to_idx = {}
    vocab = []
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    i = 0
    for word in tokenizer.word_index:
        if word in embeddings_index:
            W[i] = embeddings_index[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx
pretrained_weights, vocab, vocab2index = get_emb_matrix(embeddings_index, vocab_size)

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Adjacency Matrix

In [27]:
adj_matrix_path = 'adjacency_matrix.json'
# Count all labels.
nums = np.sum(np.array(df_tags), axis=0)
adj = np.zeros((len_tags, len_tags), dtype=int)
# Now iterate over the whole training set and consider all pairs of labels in sample annotation.
for sample in np.array(df_tags):
    sample_idx = np.argwhere(sample > 0)[:, 0]
    # We count all possible pairs that can be created from each sample's set of labels.
    for i, j in itertools.combinations(sample_idx, 2):
        adj[i, j] += 1
        adj[j, i] += 1
# Save it for further use.        
with open(adj_matrix_path, 'w') as fp:
    json.dump({
        'nums': nums.tolist(),
        'adj': adj.tolist()
    }, fp, indent=3)

In [28]:
# GCNConv.preprocess(adj).astype('f4')
tag_embeddings.shape

(723, 100)

### Defining Model

In [96]:
# def custom_model(length, vocab_size, tag_embeddings, adj):
#     inputs1 = Input(shape=(length, ))
#     embedding = Embedding(vocab_size, 100)(inputs1)
#     conv = Conv1D(filters=100, kernel_size=4, activation='relu')(embedding)
#     pool = GlobalMaxPooling1D()(conv)
#     model = Model(inputs1, pool)
#     print(model.summary())
    
#     X_in = Input(shape=(len(tag_embeddings[0]), ))
# #     fltr_in = Input((len(tag_embeddings), ))
      
#     graph_conv_1 = GCNConv(50, activation='relu')([X_in, adj])
#     graph_conv_2 = GCNConv(50, activation='relu')([graph_conv_1, adj])
#     print(pool.shape)
#     print(graph_conv_2.shape)
#     output = tf.matmul(pool, tf.transpose(graph_conv_2))
#     output = sigmoid(output)
#     print(output.shape)
    
#     gcn = Model(inputs = [inputs1, X_in], outputs = output)
#     print(gcn.summary())
#     gcn.compile(optimizer='adam', loss='binary_crossentropy', weighted_metrics=['acc'])
#     gcn.fit([X, tag_embeddings] , y.values, epochs=10)
# custom_model(length, vocab_size, tag_embeddings, adj)

In [30]:
print(length)
print(vocab_size)

3325
24280


In [31]:
tag_embeddings.shape

(723, 100)

### Pytorch Implementation

In [32]:
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda')
tag_embeddings = torch.from_numpy(tag_embeddings)

In [33]:
#dataloader class for batch training
class QuestionDataset(Dataset):
    def __init__(self, X, y):
        #data loading
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        #self.tag_embeddings = torch.from_numpy(tag_embeddings)
        self.n_samples = X.shape[0]
    def __getitem__(self, item):
        #dataset[0]
        return self.X[item], self.y[item]
    def __len__(self):
        #le(dataset)
        return self.n_samples
# data = QuestionDataset()
# a, b, c = data[0]
# print(a.shape, b.shape, c.shape)

In [108]:
# function for creating adjacency matrix (over-smoothing problem is also taken into consideration)
'''
num_classes ---> no. of unique tags
t ---> threshold for binarizing the correlation matrix.
p ---> determines the weights assigned to a node itself and other correlated nodes
adj_data ---> contains co-occurrence matrix and corresponding tag counts
    
*Note -> for info. regarding all these parameters go to https://arxiv.org/pdf/1904.03582.pdf
'''

t = 0.4
p = 0.2
def gen_A(len_tags, t, p, adj_data):
    adj = np.array(adj_data['adj']).astype(np.float32)
    nums = np.array(adj_data['nums']).astype(np.float32)
    nums = nums[:, np.newaxis]
    adj = adj / nums
    adj[adj < t] = 0
    adj[adj >= t] = 1
    adj = adj * p / (adj.sum(0, keepdims=True) + 1e-6)  
    adj = adj + np.identity(len_tags, np.int)
    return adj

#normalization of Adjacency matrix
def gen_adj(A):
    D = torch.pow(A.sum(1).float(), -0.5)
    D = torch.diag(D).type_as(A)
    adj = torch.matmul(torch.matmul(A, D).t(), D)
    return adj

# Use threshold to define predicted labels and invoke sklearn's metrics with different averaging strategies.
def calculate_metrics(pred, target, threshold=0.5):
    pred = np.array(pred > threshold, dtype=int)
    return {
            'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }

In [102]:
#defining Graph Convolution Network
"""
Simple GCN layer (pytorch), similar to https://arxiv.org/abs/1609.02907

"""
class GraphConvolution(nn.Module):
    
    def __init__(self, in_features, out_features, bias=False):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(in_features, out_features), requires_grad=True)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(1, 1, out_features), requires_grad=True)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input.float(), self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class TagRetrieval(nn.Module):
    
    def __init__(self, len_tags, adj_matrix_path, vocab_size, embedding_dim, glove_weights, t=0.1, p=0.25):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(glove_weights))
        self.conv1d = nn.Conv1d(embedding_dim, embedding_dim, 4, padding=0)
        #self.globalmaxpool1d = nn.AdaptiveMaxPool2d(output_size=(embedding_dim, 1))
#         self.flatten = nn.Flatten()
#         self.linear = nn.Linear(length*embedding_dim, 50)
        self.dropout = nn.Dropout(0.5)
        self.gc1 = GraphConvolution(embedding_dim, embedding_dim)
        self.gc2 = GraphConvolution(embedding_dim, embedding_dim)
        self.relu = nn.LeakyReLU(0.2)
        self.sigm = nn.Sigmoid()
        # Load data for adjacency matrix
        with open(adj_matrix_path) as fp:
            adj_data = json.load(fp)
        # Compute adjacency matrix
        adj = gen_A(len_tags, t, p, adj_data)
        self.A = nn.Parameter(torch.from_numpy(adj).float(), requires_grad=False)
        
    def forward(self, question, tags_embeddings):
        # Get visual features from image
        #feature = self.features(imgs)
        #feature = feature.view(feature.size(0), -1)
        embedding = self.embedding(question)
        embedding = embedding.permute([0, 2, 1])
#         embedding = self.dropout(embedding)
        conv1d = self.dropout(embedding)
        conv1d = self.conv1d(embedding)
#         globalmaxpool1d = self.globalmaxpool1d(conv1d)
#         feature = globalmaxpool1d.squeeze()
        conv1d = conv1d.permute([0, 2, 1])
        feature, _  = torch.max(conv1d, 1)
#         feature = self.linear(self.flatten(embedding))
        # Get graph features from graph
        #inp = inp[0].squeeze()
        inp = tag_embeddings
        adj = gen_adj(self.A).detach()
        x = self.gc1(inp, adj)
        x = self.relu(x)
        x = self.gc2(x, adj)
        
        # We multiply the features from GСN and СNN in order to take into account 
        # the contribution to the prediction of classes from both the text and the graph.
        x = x.transpose(0, 1)
        x = torch.matmul(feature, x)
        return self.sigm(x)
    
model = TagRetrieval(len_tags, adj_matrix_path, vocab_size, 100, pretrained_weights)
model = model.to(device)
print(model.parameters)

<bound method Module.parameters of TagRetrieval(
  (embedding): Embedding(24280, 100, padding_idx=0)
  (conv1d): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (gc1): GraphConvolution (100 -> 100)
  (gc2): GraphConvolution (100 -> 100)
  (relu): LeakyReLU(negative_slope=0.2)
  (sigm): Sigmoid()
)>


In [103]:
batch_size = 32
num_workers = 8
lr = 0.001
max_epoch_number = 100

In [104]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

In [105]:
train_dataset = QuestionDataset(X_train, y_train)
val_dataset = QuestionDataset(X_val, y_val)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=num_workers)
num_train_batches = int(np.ceil(train_dataset.__len__()/ batch_size))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [106]:
for epoch in range(max_epoch_number):
    batch_losses = []
    for batch_number, (X, y) in enumerate(train_dataloader):
        X, y, tag_embeddings = X.to(device), y.to(device), tag_embeddings.to(device)
        optimizer.zero_grad()
        model_result = model(X.long(), tag_embeddings)
        loss = criterion(model_result, y.float())
        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()
        batch_losses.append(batch_loss_value)
        if (batch_number+1)%50==0:
            print(f'epoch: {epoch+1}/{max_epoch_number}, step: {batch_number+1}/{num_train_batches}, loss: {batch_loss_value}')
    loss_value = np.mean(batch_losses)
    print("epoch:{:2d} loss:{:.3f}".format(epoch, loss_value))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: 1/100, step: 50/250, loss: 0.04172871261835098
epoch: 1/100, step: 100/250, loss: 0.03510912507772446
epoch: 1/100, step: 150/250, loss: 0.031027497723698616
epoch: 1/100, step: 200/250, loss: 0.034056104719638824
epoch: 1/100, step: 250/250, loss: 0.031450215727090836
epoch: 0 loss:0.043
epoch: 2/100, step: 50/250, loss: 0.024928493425250053
epoch: 2/100, step: 100/250, loss: 0.02843504771590233
epoch: 2/100, step: 150/250, loss: 0.026775911450386047
epoch: 2/100, step: 200/250, loss: 0.027312731370329857
epoch: 2/100, step: 250/250, loss: 0.027582045644521713
epoch: 1 loss:0.027
epoch: 3/100, step: 50/250, loss: 0.024381250143051147
epoch: 3/100, step: 100/250, loss: 0.025468576699495316
epoch: 3/100, step: 150/250, loss: 0.023530077189207077
epoch: 3/100, step: 200/250, loss: 0.027302557602524757
epoch: 3/100, step: 250/250, loss: 0.024640686810016632
epoch: 2 loss:0.025
epoch: 4/100, step: 50/250, loss: 0.022720878943800926
epoch: 4/100, step: 100/250, loss: 0.02478722296655

In [40]:
# for batch_number, (X, y) in enumerate(dataloader):
#     print(batch_number, X.shape, y.shape, tag_embeddings.shape)

In [109]:
# with torch.no_grad():
#     result = calculate_metrics(np.array(model_result.cpu()), np.array(y.cpu()))
#     print(result)
with torch.no_grad():
  model_result = []
  targets = []
  for X, y in train_dataloader:
    X, y, tag_embeddings = X.to(device), y.to(device), tag_embeddings.to(device)
    model_batch_result = model(X.long(), tag_embeddings)
    model_result.extend(model_batch_result.cpu().numpy())
    targets.extend(y.cpu().numpy())
result = calculate_metrics(np.array(model_result), np.array(targets))
print(result)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'micro/precision': 0.9930734309917162, 'micro/recall': 0.9386861313868613, 'micro/f1': 0.9651141635586281, 'macro/precision': 0.9438293329131467, 'macro/recall': 0.8717104990585849, 'macro/f1': 0.8994698317506449, 'samples/precision': 0.9888714781746032, 'samples/recall': 0.9460333333333333, 'samples/f1': 0.9607086198523698}


In [110]:
with torch.no_grad():
  model_result = []
  targets = []
  for X, y in val_dataloader:
    X, y, tag_embeddings = X.to(device), y.to(device), tag_embeddings.to(device)
    model_batch_result = model(X.long(), tag_embeddings)
    model_result.extend(model_batch_result.cpu().numpy())
    targets.extend(y.cpu().numpy())
  result = calculate_metrics(np.array(model_result), np.array(targets))
  print(result)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'micro/precision': 0.3699851411589896, 'micro/recall': 0.08749121574139143, 'micro/f1': 0.14151747655583977, 'macro/precision': 0.061947860492459324, 'macro/recall': 0.019415298521588745, 'macro/f1': 0.027183688286331435, 'samples/precision': 0.15826206016206015, 'samples/recall': 0.09771666666666667, 'samples/f1': 0.10688347208347207}


In [ ]:
# pred = np.array(model_result > 0.5, dtype=int)
# print(f1_score(y_true=np.array(y), y_pred=pred, average = 'micro'))
while True:pass

In [46]:
x = torch.tensor([[1,2, 12,34, 56,78],
                 [12,45, 99,67, 6, 54],
                 [3,24, 6,99, 12,56]])
embed = nn.Embedding(3325, 10, padding_idx=0)
print(x.shape)
x = embed(x)
print(x.shape)
# print(x[0])
# x = x.permute([0, 2, 1])
# print(x.shape)
# conv1d = nn.Conv1d(50, 2048, 4, padding=0)
# x = conv1d(x)
# print(x.shape)
# x = x.permute([0, 2, 1])
# print(x.shape)
# x, _  = torch.max(x, 1)
# print(x.shape)
# globalmaxpool1d = nn.AdaptiveMaxPool2d(output_size=(50, 1))
# x = globalmaxpool1d(x)
# print(x.shape)
# x = x.squeeze()
# print(x.shape)


# gc1 = GraphConvolution(50, 1024)
# y = gc1(torch.from_numpy(tag_embeddings), torch.from_numpy(A).float())
# print(y.shape)
# gc2 = GraphConvolution(1024, 2048)
# y = gc2(y, torch.from_numpy(A).float())
# print(y.shape)
# y = y.transpose(0, 1)
# print(y.shape)
# y = torch.matmul(x, y)
# print(y.shape)
# sigm = nn.Sigmoid()
# y = sigm(y)
# print(y.shape)


torch.Size([3, 6])
torch.Size([3, 6, 10])


In [45]:
with open(adj_matrix_path) as fp:
    adj_data = json.load(fp)
A = gen_A(723, t, p, adj_data)
A.shape

(723, 723)

KeyboardInterrupt: ignored